In [ ]:
import blocksDB_diagramming as BDD

BDD.load_dependencies()

NAME_PREFIX="TEMP"

DBFILE,FROM_BLOCK,TO_BLOCK ="temp1.db", 54, 124
#DBFILE,FROM_BLOCK,TO_BLOCK ="temp1.db", 0, 233

#DBFILE,FROM_BLOCK,TO_BLOCK ="temp2.db", 1, 5000

imgpath="img"

In [ ]:
conn, blocknumbers = BDD.load_db_and_check_complete(DBFILE)
BDD.simple_stats(conn)
df = BDD.read_whole_table_into_dataframe(conn)
conn.close()

assert BDD.check_timestamp_format(df)

BDD.add_columns(df)
BDD.show_peak_TPS(df)
    
# if FROM_BLOCK==-1: FROM_BLOCK = min(blocknumbers)[0]
# if TO_BLOCK==-1: TO_BLOCK = max(blocknumbers)[0]

# N.B.: Redefined: FROM_BLOCK, TO_BLOCK now experiment limits!

print (FROM_BLOCK, TO_BLOCK)



##### TODO: Check whether any of these is start position dependent!


In [ ]:
print(df.index)
print (max(df.index))

In [ ]:
def experiment_slice(df, FROM_BLOCK, TO_BLOCK, emptyBlocks):
    """
    cut out the dataframe from FROM_BLOCK to TO_BLOCK+emptyBlocks (incl that last one)
    can handle that df starts not at block 0
    can handle that limits are smaller or larger than available blocknumbers
    
    """
    assert FROM_BLOCK <= TO_BLOCK
    
    index_from = min( df[df['blocknumber'] >= FROM_BLOCK].index.tolist() )
    # print (slice_from)
        
    index_to = max( df[df['blocknumber'] <= TO_BLOCK+emptyBlocks].index.tolist() )
    # print(slice_to)
        
    dfs = df[index_from:index_to + 1]
    
    return dfs, index_from, index_to

# dfs, slice_from, slice_to = experiment_slice(df, 50, 50, 0)
dfs, index_from, index_to = experiment_slice(df, FROM_BLOCK, TO_BLOCK, 10)
print (index_from, index_to)
dfs

In [ ]:
print ("Old diagrams could not correctly handle the blocknumbers unless dataframe started with block zero")
print ("See title when using temp1.db which starts at block 30 only")
print ("But this is obsolete now anyways.")

fn = BDD.diagrams_oldversion(df, index_from, index_to, NAME_PREFIX, gas_logy=True, bt_logy=True, imgpath=imgpath)

print (fn)



In [ ]:
dfs, index_from, index_to = experiment_slice(df, FROM_BLOCK, TO_BLOCK, 10)
print (index_from, index_to)
dfs

In [ ]:
import matplotlib.pyplot as plt
fig, axes = plt.subplots(2, 2, figsize=(16,9))

In [ ]:
print (dfs.columns)
print (dfs[['blocknumber','txcount','timestamp']].head(5))
print (dfs[['blocknumber','txcount','timestamp']].tail(15))

In [ ]:
def averageTps_wholeExperiment_viaLeavingOutEmptyBlocks(dfs, emptyBlocks):
    
    ts1 = dfs.iloc[0]['timestamp'] # stop clock starts WHEN block 0 is in already!
    ts2 = dfs.iloc[-emptyBlocks-1]['timestamp']   # and ends at last filled block
    # print (ts1, ts2)
    
    txs=sum(dfs['txcount'][1:-emptyBlocks+1]) # N.B.: start summing at block 1 not 0 !
    # print (txs)
    
    tps=(txs/(ts2-ts1))
    # print (tps)
    return tps, "%.1f" % tps

averageTps_wholeExperiment_viaLeavingOutEmptyBlocks(dfs, emptyBlocks=10)

In [ ]:
def averageTps_wholeExperiment(dfs, FROM_BLOCK, TO_BLOCK):
    """
    works on already sliced dataframe, 
    where first experiment block is index 0
    and last experiment block is index [TO_BLOCK - FROM_BLOCK]
    N.B.:
    we cannot rely on the blocktime of very first block
    so we simply leave the transactions out of the summation, and 
    the duration is from when that first block WAS MINED = its timestamp.
    """
    
    blocks = TO_BLOCK - FROM_BLOCK + 1
    ts1 = dfs.iloc[0]['timestamp'] # stop clock starts WHEN block 0 is in already!
    ts2 = dfs.iloc[blocks-1]['timestamp']   # and ends at last filled block
    duration = ts2-ts1
    print (ts1, ts2, duration)
    
    txs=sum(dfs['txcount'][1:blocks]) # N.B.: start summing at block 1 not 0 !
    print (txs)
    
    tps=(txs/duration)
    print (tps)
    return tps, "%.1f" % tps

averageTps_wholeExperiment(dfs, FROM_BLOCK, TO_BLOCK)

In [ ]:
def averager(dfs, col, emptyBlocks, fmt="%.1f"):
    """
    We want the real average of that 'col', taken only over the non-empty blocks.
    N.B.: this assumes that there are actually enough emptyBlocks at the end!
    """
    av = dfs[col] [:-emptyBlocks] .mean()
    avTxt = fmt % av
    return av, avTxt


def avgLine(ax, dfs, emptyBlocks, avg, avgTxt):
    """
    horizontal line plus text on white background
    """
    blMin, blMax = min(dfs["blocknumber"])+1, max(dfs["blocknumber"][:-emptyBlocks])
    ax.plot([blMin, blMax], [avg, avg], "k-")
    
    ax.text(blMin + (blMax-blMin + emptyBlocks)*0.95, avg, avgTxt, 
            bbox=dict(facecolor='white', edgecolor='white'))
    

def axes_simplifier(ax, logYscale=False):
    """
    otherwise matplotlib automatically switches on notations on the ticks 
    that might be confusing to non-technical people
    """
    ax.get_xaxis().get_major_formatter().set_useOffset(False)
    ax.get_xaxis().get_major_formatter().set_scientific(False)
    if not logYscale:
        ax.get_yaxis().get_major_formatter().set_useOffset(False)
        ax.get_yaxis().get_major_formatter().set_scientific(False)

        
def tps_plotter(ax, dfs, FROM_BLOCK, TO_BLOCK, emptyBlocks):
    """
    TPS average calculated only over non-empty blocks!
    average calculated for TPS (not for smoothed 3, 5, 10 blocks averages)
    N.B.: this assumes that there are actually enough emptyBlocks at the end!
    """
    
    cols=['TPS_1blk', 'TPS_3blks', 'TPS_5blks', 'TPS_10blks']
    for col in cols:
        ax.plot(dfs['blocknumber'], dfs[col])

    axes_simplifier(ax)
   
    #avg, avgTxt = averager(dfs, cols[0], emptyBlocks, "%.1f")
    #legend = [cols[0] + " (avg %s)"%avgTxt ] + cols[1:]
    ax.legend(cols); 
   
    avg2, avgTxt2 = averageTps_wholeExperiment(dfs, FROM_BLOCK, TO_BLOCK)
    avgLine(ax, dfs, emptyBlocks, avg2, avgTxt2)
    
    ax.set_title("avg TPS %s = #TX whole experiment / blocktimes diff" % avgTxt2)
  
    
# fig, axes = plt.subplots(1, 1, figsize=(16,9))
fig, axes = plt.subplots(1, 1, figsize=(6,4))
tps_plotter(axes, dfs, FROM_BLOCK, TO_BLOCK, 10)

In [ ]:
def blocktimes_plotter(ax, dfs):
    "show the blocktimes"

    ax.set_title("blocktime seconds since last block")
    
    ax.scatter(x=dfs['blocknumber'], y=dfs['blocktime'], c="b", marker="x")
    
    axes_simplifier(ax)

fig, axes = plt.subplots(1, 1, figsize=(16,9))
# fig, axes = plt.subplots(1, 1, figsize=(6,4))
blocktimes_plotter(axes, dfs)    

In [ ]:
def blocksizes_plotter(ax, dfs, emptyBlocks):
    # blocksizes
    # plus average line
    
    ax.scatter(dfs['blocknumber'], dfs['size'], c="g", marker="o")
    ax.plot(   dfs['blocknumber'], dfs['size'], "g-")
    
    avg, avgTxt = averager(dfs, 'size', emptyBlocks, "%d")
    avgLine(ax, dfs, emptyBlocks, avg, avgTxt)
    
    ax.set_title("blocksizes in bytes")
    
    axes_simplifier(ax)

#fig, axes = plt.subplots(1, 1, figsize=(16,9))
fig, axes = plt.subplots(1, 1, figsize=(6,4))
blocksizes_plotter(axes, dfs, 15)    

In [ ]:
# just a test what happens when many many blocks

import pandas as pd
testdf = pd.DataFrame({"blocknumber" : range(100000,300000)})
testdf["size"]=40000
testdf
fig, axes = plt.subplots(1, 1, figsize=(6,4))
blocksizes_plotter(axes, testdf, 10)

In [ ]:
?plt.scatter

In [ ]:
?plt.plot

In [ ]:
?axes.text

In [ ]:
dfs['GLPS_1blk']

In [ ]:
def gas_plotter(ax, dfs):
    ax.set_title("gasUsed and gasLimit per second")    
    
    ax.plot(   dfs['blocknumber'], dfs['GLPS_1blk']) # , "g-")
    ax.plot(   dfs['blocknumber'], dfs['GUPS_1blk']) # 
    
    ax.set_yscale('log')
    
    #ax=df[['blocknumber', 'GLPS_1blk', 'GUPS_1blk']][blockFrom:blockTo].plot(x='blocknumber', 
    #                                                                         rot=90, ax=axes[1,1], 
    #                                                                         logy=gas_logy)
    
    axes_simplifier(ax, logYscale=True)
    ax.legend (["gasLimit/sec", "gasUsed/sec"] )

fig, axes = plt.subplots(1, 1, figsize=(6,4))
gas_plotter(axes, dfs)

In [ ]:
def diagrams(prefix, df, blockFrom, blockTo, emptyBlocks=10):
    """
    new version more precise & consistent
    * slice of whole experiment (from/to), plus some emptyBlocks at the end
    * averages are calc'ed over the experiment blocks only! 
    * average lines & number for tps & block size
    * title shows more infos about experiment
    * x-axis ticks issues solved
    """
    
    # offset=min(df["blocknumber"])
    # just the slice of the experiment + 10 extra blocks:
    # dfs = df[FROM_BLOCK-offset:TO_BLOCK-offset+emptyBlocks+1] 
    dfs, index_from, index_to = experiment_slice(df, blockFrom, blockTo, emptyBlocks)
    
    # https://github.com/matplotlib/matplotlib/issues/5907#issuecomment-179001811
    import matplotlib
    matplotlib.rcParams['agg.path.chunksize'] = 10000
    
    fig, axes = plt.subplots(2, 2, figsize=(16,9)) #,  sharex=True)
    fig.subplots_adjust(hspace=0.25, wspace=0.20)
    
    tps_plotter(axes[0,0], dfs, FROM_BLOCK, TO_BLOCK, emptyBlocks)
    blocktimes_plotter(axes[0,1], dfs)
    blocksizes_plotter(axes[1,0], dfs, emptyBlocks)  
    gas_plotter(axes[1,1], dfs)
    
    txs=sum(dfs['txcount'][0:-emptyBlocks+1])
    title = prefix + " blocks %d-%d with %d txs ~ %d txs/block" 
    title = title % (blockFrom, blockTo, txs, round(txs/(blockTo-blockFrom+1)))
    fig.suptitle(title, fontsize=16)
    
    return fig, axes, dfs, txs
    
print (FROM_BLOCK, TO_BLOCK)
_, _, dfs, txs = diagrams("TEMP", df, FROM_BLOCK, TO_BLOCK, 12)
print (min(dfs["blocknumber"]), max(dfs["blocknumber"]))